In [99]:
"""
Import our files as a data frames.
"""
import re
import pandas as pd

female_male_df = pd.read_csv("female_male_ratio-NYC.csv")

subway_address_df = pd.read_csv("http://web.mta.info/developers/data/nyct/subway/Stations.csv")

In [100]:
"""
Create single string from GTFS Lat/Lons
"""

subway_address_df['GTFS Latitude'] = subway_address_df['GTFS Latitude'].map(str)
subway_address_df['GTFS Longitude'] = subway_address_df['GTFS Longitude'].map(str)

subway_address_df['lat_lon_combined'] = subway_address_df['GTFS Latitude'] + ", " + subway_address_df["GTFS Longitude"]


In [101]:
"""
Connect subway data to a certain zip codes using geo-locator.
"""
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="7551")

subway_address_df['location'] = subway_address_df['lat_lon_combined'].apply(geolocator.reverse)
subway_location_df = subway_address_df.copy()

In [102]:
"""
Convert the series of location objects to a zipcode.
"""

subway_location_df['zipcode'] = subway_location_df['location'].apply(lambda loc: loc.address.split(',')[-2].strip())

In [140]:
"""
Merge the data between female/male ratio and subway station on zipcode
"""
female_male_df.rename(columns = {'Zip Code':'zipcode','Location': 'female_latlon', 'Females / Male':'female_male_ratio'},inplace=True)
female_male_df['zipcode'] = female_male_df['zipcode'].map(str)

female_subway_df = pd.merge(female_male_df,subway_location_df,on='zipcode',how='left')
female_subway_df.sort_values('female_male_ratio', ascending = False).head()

,zipcode,female_latlon,City,Population,female_male_ratio,Station ID,Complex ID,GTFS Stop ID,Division,Line,...,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Notes,lat_lon_combined,location
0,10037,"40.813159, -73.937352","New York, New York","16,984",1.42,393.0,393.0,622,IRT,Lexington Av,...,6,Subway,40.798629,-73.94161700000001,Uptown & The Bronx,Downtown,0.0,NaN,"40.798629, -73.94161700000001","(116th Street, Lexington Avenue, East Harlem, ..."
2,10037,"40.813159, -73.937352","New York, New York","16,984",1.42,398.0,398.0,627,IRT,Lexington Av,...,6,Subway,40.77362,-73.959874,Uptown & The Bronx,Downtown,0.0,NaN,"40.77362, -73.959874","(77th Street, Lexington Avenue, Lenox Hill, Ma..."
3,10037,"40.813159, -73.937352","New York, New York","16,984",1.42,400.0,613.0,629,IRT,Lexington Av,...,4 5 6,Subway,40.762526,-73.967967,Uptown & The Bronx,Downtown & Brooklyn,0.0,NaN,"40.762526, -73.967967","(International Plaza, 750, Lexington Avenue, U..."
4,10037,"40.813159, -73.937352","New York, New York","16,984",1.42,401.0,612.0,630,IRT,Lexington Av,...,6,Subway,40.757107,-73.97192,Uptown & The Bronx,Downtown,1.0,NaN,"40.757107, -73.97192","(51st Street, Lexington Avenue, Midtown East, ..."
5,10037,"40.813159, -73.937352","New York, New York","16,984",1.42,437.0,437.0,302,IRT,Lenox - White Plains Rd,...,3,Subway,40.820421,-73.936245,148 St,Downtown & Brooklyn,0.0,NaN,"40.820421, -73.936245",(Lenox Avenue & 145th Street at Southeast Corn...


In [141]:
"""
Create a reduced dataframe with just the pertinent information
"""
reduced_subway_fem_df = pd.DataFrame({'station_id': female_subway_df['Station ID'],'Stop Name': female_subway_df['Stop Name'],'zipcode':female_subway_df['zipcode'],'female_male_ratio':female_subway_df['female_male_ratio']})
reduced_subway_fem_df.sort_values('female_male_ratio',ascending=False).head()


,station_id,Stop Name,zipcode,female_male_ratio
0,393.0,116 St,10037,1.42
2,398.0,77 St,10037,1.42
3,400.0,59 St,10037,1.42
4,401.0,51 St,10037,1.42
5,437.0,145 St,10037,1.42


In [143]:
"""
Create a CSV from this map
"""
reduced_subway_fem_df.to_csv('female_male_ratio_subway_mapping.csv',index=False)
